In [2]:
%matplotlib qt
import numpy as np
from scipy.spatial.transform import Rotation as spicyrot
from orix.quaternion import Orientation
import numpy as np
from orix.vector import Miller, Vector3d
from orix import plot
from orix.crystal_map import Phase
import matplotlib.pyplot as plt
from matplotlib import gridspec
from orix.io import load
import os
from orix.crystal_map import  Phase

from diffpy.structure import Lattice, Structure
from orix.vector import Miller

In [3]:
class ScipyRotation:
    def __init__(self,vectorpair1,vectorpair2) -> None:
        self.vectorpair1 = vectorpair1
        self.vectorpair2 = vectorpair2
    

    def get_full_rotation_matrix(self):
        return spicyrot.align_vectors(self.vectorpair1, self.vectorpair2)

In [4]:
def get_arr(file):
    return np.genfromtxt(file, delimiter=",")

In [5]:
#the following path contains files which include information from SAD patterns
#such as indexing and gonjo tilt

measurments_path = "/Users/anders/Library/CloudStorage/OneDrive-NTNU/Prosjektoppgave/"\
                "Data/For_align_vectors/measurements"

In [6]:
class GonjoPosition:
    """Class represents a gonjo defined by the x and y tilt"""
    def __init__(self, x_tilt, y_tilt, degrees=True) -> None:
        if degrees:
            self.x_tilt = np.radians(x_tilt)
            self.y_tilt = np.radians(y_tilt)


In [7]:
class RotationGenerator:
    """Class for generating rotation matrices converting from one gonjo to another"""
    def __init__(
        self,
        new_gonjo_pos: GonjoPosition,
        old_gonjo_pos: GonjoPosition = GonjoPosition(0, 0),
    ) -> None:
        self.new_gonjo_pos = new_gonjo_pos
        self.old_gonjo_pos = old_gonjo_pos

        self.alpha_0 = self.old_gonjo_pos.x_tilt
        self.beta_0 = self.old_gonjo_pos.y_tilt

        self.alpha = self.new_gonjo_pos.x_tilt - self.old_gonjo_pos.x_tilt
        self.beta = self.new_gonjo_pos.y_tilt - self.old_gonjo_pos.y_tilt

    def get_T1(self) -> np.ndarray:
        a11 = 1
        a12 = 0
        a13 = 0
        a21 = 0
        a22 = np.cos(self.alpha)
        a23 = -np.sin(self.alpha)
        a31 = 0
        a32 = np.sin(self.alpha)
        a33 = np.cos(self.alpha)

        return np.array([[a11, a12, a13], 
                         [a21, a22, a23],
                         [a31, a32, a33]])

    def get_T2(self) -> np.ndarray:
        a11 = np.cos(self.beta)
        a12 = -np.sin(self.beta) * np.sin(self.alpha_0)
        a13 = np.sin(self.beta) * np.cos(self.alpha_0)
        a21 = np.sin(self.beta) * np.sin(self.alpha_0)
        a22 = np.cos(self.alpha_0) ** 2 + np.sin(self.alpha_0) ** 2 * np.cos(self.beta)
        a23 = np.sin(self.alpha_0) * np.cos(self.alpha_0) * (1 - np.cos(self.beta))
        a31 = -np.sin(self.beta) * np.cos(self.alpha_0)
        a32 = np.sin(self.alpha_0) * np.cos(self.alpha_0) * (1 - np.cos(self.beta))
        a33 = np.sin(self.alpha_0) ** 2 + np.cos(self.alpha_0) ** 2 * np.cos(self.beta)

        return np.array([[a11, a12, a13], 
                         [a21, a22, a23], 
                         [a31, a32, a33]])

    def get_full_rotation_matrix(self) -> np.ndarray:
        return self.get_T1() @ self.get_T2()

In [8]:
def inplane_rotation_by(vec, theta = 90-34.56):
    rot_vec = np.array([[0,0,1]],dtype = float)
    rot_vec = rot_vec * np.radians(theta)
    rotation_matrix = spicyrot.from_rotvec(rot_vec)

    return (rotation_matrix.as_matrix() @ vec.T)[0]

In [9]:
def find_normal_to_hex_plane(hex_plane: np.array,  a, c) -> np.array:
    h, k, l = hex_plane[:]
    n1 = 2*h+k
    n2 =  2*k+h
    n3 = 3/2 *a**2/c**2 *l
    return np.array([n1, n2, n3])


In [10]:
def convert_hkl(h,k,l, a = 6.115, c = 11.41):

    #convert to plane normal vector
    u,t,w = find_normal_to_hex_plane(np.array([h,k,l]), a,c)

    lattice = Lattice(a, a, c, 90,90,120)
    phase = Phase(point_group="6mm", structure=Structure(lattice=lattice))

    #convert plane normal to cartesian coords.
    abcr = Miller(uvw=[[u,t,w]], phase=phase)
    x,y,z = abcr._data[0]

    return  y,x,z


In [11]:
class DiffractionObject:
    def __init__(self, area_number, symmetry) -> None:
        self.area = f"Area{area_number}"
        self.symmetry = symmetry
        self.dataarr = get_arr(os.path.join(measurments_path,self.area + ".csv"))
        self.millers = []
        self.vectors = []

        temp_vectors = []
        for line in self.dataarr[1:,:]:
            angle = line[4]
            lenght = line[5]
            x,y,z = convert_hkl(*line[6:9])
            
            x_tilt,y_tilt = line[9:]

            x_comp = lenght * np.cos(np.radians(angle))
            y_comp = lenght * np.sin(np.radians(angle))
            
            self.millers.append([x,y,z])
            temp_vectors.append([x_comp,y_comp,0])
      
        self.millers = np.array(self.millers)
        temp_vectors = np.array(temp_vectors)
        for vector,miller in zip(temp_vectors, self.millers):
            vector /= np.linalg.norm(vector)
            vector *= np.linalg.norm(miller)
            self.vectors.append(vector)
        self.vectors = np.array(self.vectors)
    
        self.gonjo = GonjoPosition(x_tilt,y_tilt)

In [12]:
def get_ori_in_reference_tilt(self,
                            reference : GonjoPosition, 
                            inplane = 0) -> Orientation:
    gonjo_rot_generator = RotationGenerator(reference, self.gonjo)
    new_vectors = np.zeros(self.vectors.shape)
    for i,vector in enumerate(self.vectors):

        vector = inplane_rotation_by(vector,90-34.56)
        vector = gonjo_rot_generator.get_full_rotation_matrix() @ vector
        vector = inplane_rotation_by(vector, 34.56-90)
        vector = inplane_rotation_by(vector,inplane)
        
        new_vectors[i,:] = vector
    
    self.vectors = new_vectors
    rotgen = ScipyRotation(self.millers,self.vectors)

    rotation, error = rotgen.get_full_rotation_matrix()
    print(f"The alignment error for {self.area} is {error :.2f}")
    
    return Orientation.from_matrix(rotation.as_matrix(), self.symmetry)

DiffractionObject.get_ori_in_reference_tilt = get_ori_in_reference_tilt

In [1]:
def get_SPED_ori_from_area(area, plot_map = False):
    xmap= load("//Users/anders/Library/CloudStorage/OneDrive-NTNU/Prosjektoppgave/"\
    "Data/Processed/One_deg/error_fixed/ErMnO3_1_deg_scan_errors_fixed_rotated.hdf5")
    area_to_coords_dict = {
        "Area1" : [45,504],
        "Area2" : [45,440],
        "Area3" : [201,477],
        "Area4" : [140,595],
        "Area5" : [127,540],
        "Area6" : [170,554],
        "Area7" : [185,390],
        "Area8" : [250,370],
        "Area9" : [220,510],
        "Area10" : [55,287],
        "Area11" : [57,135],
        "Area12" : [160,175],
        "Area13" : [256,52],
        "Area14" : [280,204],
        "Area15" : [51,619],
        "Area16" : [206,220],
        "Area17" : [330,50],
    }
    if plot_map:
        symmetry = Phase("6mm", 185).point_group.laue
        ipfkey = plot.IPFColorKeyTSL(symmetry)
        ipfkey.direction  = Vector3d.zvector()
        color =  ipfkey.orientation2color(xmap["ErMnO3"].orientations)
        xmap["ErMnO3"].plot(color)
        for key,value in area_to_coords_dict.items():
            
            plt.scatter(*value[::-1],label = key)
        plt.legend()
    x,y =area_to_coords_dict[f"Area{area}"]
    
    return xmap[x,y].orientations


In [16]:

def main(num, start = 0, inplane = 0):
    symmetry = Phase("6mm", 185).point_group.laue
    reference_gonjo = GonjoPosition(2.8,0.4)
 
    angles = []

    for i in range(num):
        rotation = inplane
        area = i+1 + start

        if area in [3,6,12,13,14,15,17]: #rotate some because of 180° disambiguity
            rotation += 180
        
        diff_obj = DiffractionObject(area, symmetry)

        ori_manual = diff_obj.get_ori_in_reference_tilt(reference_gonjo, rotation)
        ori_sped = get_SPED_ori_from_area(area)
        
        ori_sped.symmetry = symmetry
        angle=(np.rad2deg(ori_sped.angle_with(ori_manual)))
                
        angles.append(angle)   
    

    return np.asarray(angles)

angles = main(17,0,142)

The alignment error for Area1 is 0.06
The alignment error for Area2 is 0.08
The alignment error for Area3 is 0.06
The alignment error for Area4 is 0.03
The alignment error for Area5 is 0.06
The alignment error for Area6 is 0.11
The alignment error for Area7 is 0.03
The alignment error for Area8 is 0.04
The alignment error for Area9 is 0.09
The alignment error for Area10 is 0.12
The alignment error for Area11 is 0.02
The alignment error for Area12 is 0.11
The alignment error for Area13 is 0.13
The alignment error for Area14 is 0.03
The alignment error for Area15 is 0.01
The alignment error for Area16 is 0.08
The alignment error for Area17 is 0.08


In [14]:
for i, angle in enumerate(angles):
    print(f"Area{i+1} misangle: ", f"{angle[0] :.2f}°")

Area1 misangle:  0.92°
Area2 misangle:  1.99°
Area3 misangle:  3.04°
Area4 misangle:  11.60°
Area5 misangle:  8.80°
Area6 misangle:  2.07°
Area7 misangle:  1.73°
Area8 misangle:  10.25°
Area9 misangle:  12.98°
Area10 misangle:  6.15°
Area11 misangle:  9.52°
Area12 misangle:  8.31°
Area13 misangle:  3.05°
Area14 misangle:  5.24°
Area15 misangle:  8.01°
Area16 misangle:  4.01°
Area17 misangle:  23.07°


In [19]:
#calculate average exluding area 7 (which is ignored) and area 17 (which TM has failed)
tot_angles = 0
tot = 0
for i in range(16):
    if i == 6:
        continue
    tot_angles += angles[i][0]
    tot+=1
print(f"Averge misangle between SAD and TM: {tot_angles/tot :.2f}°")

Averge misangle between SAD and TM: 6.40°
